# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

라이브러리와 프레임워크의 가장 큰 차이는 제어 흐름에 대한 주도성이 누구에게 또는 어디에 있는가 이다.프레임워크는 전체적인 흐름을 스스로가 쥐고 있고 사용자는 그 안에서 필요한 코드를 짜넣으면 되지만 라이브러리는 사용자가 전체적인 흐름을 만들어 라이브러리를 활용하는 것이라고 할 수 있다. 

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

머신러닝은 기계를 학습시켜 인공지능을 만드는 것이고, 딥러닝은 신경망을 통해 인공지능을 만드는 것으로 딥러닝은 머신러닝안에 포함된 개념이다. 머신러닝을 학습할 데이터에 대해 인간이 이미 처리,가공을 하지만 딥러닝의 경우 컴퓨터가 스스로 학습하도록 한다. 

### Q3. 아래의 코드에 주석 달기.

In [ ]:
#필요한 패키지 import 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

device = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda가 사용 가능하면 device에 "cuda"를 저장하고 사용 가능하지 않으면 "cpu"를 저장한다.
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45) #랜덤시드값 고정 
print(device + " is available")

In [ ]:
learning_rate = 0.001 #학습률설정 
batch_size = 100 #배치사이즈 설정  
num_classes = 10 #클래스 개수 설정 
epochs = 5 # 에포크 설정 

In [ ]:
#trainset, testset 데이터 다운로드 
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True, #학습용 
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()  # 데이터를 0에서 255까지 있는 값을 0에서 1사이 값으로 변환
    ])
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False, #테스트용 
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

In [ ]:
#학습시 데이터들을 배치사이즈씩 전달하기 객체 dataloader사용(한번에 학습시 시간 오래걸림)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

# input size를 알기 위해서
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

In [ ]:
#CNN층을 쌓기 위한 class생성 
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) #합성곱층 2개 쌓기 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) #드롭층 0.25의 비율 
        self.mp = nn.MaxPool2d(2) #맥스풀링 
        self.fc1 = nn.Linear(320,100) #flat한것으로 100개로 출력 
        self.fc2 = nn.Linear(100,10)  #100개의 출력을 10개로 변경 


  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) #합성곱층 첫번째에 relu씌우기 
        x = F.relu(self.mp(self.conv2(x))) #합성곱층 두번째에 relu 씌우기
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) #flat하게 해주기 
        x = self.fc1(x) #fc1레이어에 삽입 
        x = self.fc2(x) #fc2레이어에 삽입 
        return F.log_softmax(x) #logsoftmax함수에 넣어 반환 

In [ ]:

model = ConvNet().to(device) #CNN 모델 생성 
# Cost Function과 Optimizer 선택
criterion = nn.CrossEntropyLoss().to(device) #크로스엔트로피함수 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) #optimizer로 adam선택 

In [ ]:
#epoch만큼 실행 
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() # 모든 model의 gradient 값을 0으로 설정
        hypothesis = model(data) # 모델을 forward pass해 결과값 저장 
        cost = criterion(hypothesis, target) #loss 계산 
        cost.backward()# backward 함수를 호출해 gradient 계산
        optimizer.step() # 모델의 학습 파라미터 갱신
        avg_cost += cost / len(train_loader) #평균cost값 구하기 
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:

model.eval()# evaluate mode로 전환 
with torch.no_grad(): #grad 해제 
    correct = 0
    total = 0

    for data, target in test_loader: #test하자 
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] #가장 높은 값을 갖는 인덱스를 추출
        total += len(target) 
        correct += (preds==target).sum().item() #예측값이 실제값과 같은지 비교 
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.